In [1]:
from bitwuzla import *

In [2]:
# First, create a term manager instance.
tm = TermManager()
# Create a Bitwuzla options instance.
options = Options()
# Then, enable model generation.
options.set(Option.PRODUCE_MODELS, True)
# Create a Bitwuzla instance. 
bitwuzla = Bitwuzla(tm, options)

In [3]:
bits = 2
bit_with = 2**bits
gate_number = 2
gts = [0b0110, 0b001]
# this is the sbox, it is temp used, after need generate by the true sbox
xs = [0b0011, 0b0101]
ys = [0b0100, 0b0110]

sortbv = tm.mk_bv_sort(bit_with)

In [4]:
# create the input and output variables
xs_v = [tm.mk_const(sortbv, "x{}".format(i)) for i in range(bits)]
ys_v = [tm.mk_const(sortbv, "y{}".format(i)) for i in range(bits)]

# create the constraints
for i in range(bits):
    bitwuzla.assert_formula(
        tm.mk_term(Kind.EQUAL, [xs_v[i], tm.mk_bv_value(sortbv, xs[i])])
    )
for i in range(bits):
    bitwuzla.assert_formula(
        tm.mk_term(Kind.EQUAL, [ys_v[i], tm.mk_bv_value(sortbv, ys[i])])
    )

In [5]:
# create the gate input and output
ts_v = [tm.mk_const(sortbv, "t{}".format(i)) for i in range(gate_number)]
qs_v = [tm.mk_const(sortbv, "q{}".format(i)) for i in range(gate_number * 2)]
# create the constraints for the input
for i in range(gate_number):
    input0 = [tm.mk_term(Kind.EQUAL, [qs_v[2*i], xs_v[j]]) for j in range(bits)]
    input1 = [tm.mk_term(Kind.EQUAL, [qs_v[2*i + 1], xs_v[j]]) for j in range(bits)]
    inter0 = [tm.mk_term(Kind.EQUAL, [qs_v[2*i], ts_v[j]]) for j in range(i)]
    inter1 = [tm.mk_term(Kind.EQUAL, [qs_v[2*i + 1], ts_v[j]]) for j in range(i)]
    bitwuzla.assert_formula(tm.mk_term(Kind.OR, input0 + inter0))
    bitwuzla.assert_formula(tm.mk_term(Kind.OR, input1 + inter1))

In [6]:
# need encoding for gate type
gts_v = [tm.mk_const(sortbv, "gt{}".format(i)) for i in range(gate_number)]

# create the constraints for the gate type
for i in range(gate_number):
    types = [tm.mk_term(Kind.EQUAL, [gts_v[i], tm.mk_bv_value(sortbv, gts[j])]) for j in range(len(gts))]
    bitwuzla.assert_formula(
        tm.mk_term(Kind.OR, types)
    )

In [7]:
# create the constraints for the output, it is the hard part.
one_sort = tm.mk_bv_sort(1)
one = tm.mk_bv_value(one_sort, 1)
zores = tm.mk_bv_value(sortbv, 0)
ones = tm.mk_bv_value(sortbv, 2**bit_with - 1)
for i in range(gate_number):
    # create the constraints for the output
    gt0 = tm.mk_term(Kind.BV_EXTRACT, [gts_v[i]], [0, 0])
    gt1 = tm.mk_term(Kind.BV_EXTRACT, [gts_v[i]], [1, 1])
    gt2 = tm.mk_term(Kind.BV_EXTRACT, [gts_v[i]], [2, 2])
    gt3 = tm.mk_term(Kind.BV_EXTRACT, [gts_v[i]], [3, 3])
    gt0_ul = tm.mk_term(
        Kind.ITE,
        [
            tm.mk_term(Kind.BV_ULT, [gt0, one]),
            zores,
            tm.mk_term(Kind.BV_AND, [qs_v[2 * i], qs_v[2 * i + 1]]),
        ],
    )
    gt1_ul = tm.mk_term(
        Kind.ITE,
        [
            tm.mk_term(Kind.BV_ULT, [gt1, one]),
            zores,
            qs_v[2 * i + 1],
        ],
    )
    gt2_ul = tm.mk_term(
        Kind.ITE,
        [
            tm.mk_term(Kind.BV_ULT, [gt2, one]),
            zores,
            qs_v[2 * i],
        ],
    )
    gt3_ul = tm.mk_term(
        Kind.ITE,
        [
            tm.mk_term(Kind.BV_ULT, [gt3, one]),
            zores,
            ones,
        ],
    )
    bitwuzla.assert_formula(
        tm.mk_term(
            Kind.EQUAL,
            [
                ts_v[i],
                tm.mk_term(
                    Kind.BV_XOR,
                    [
                        gt0_ul,
                        tm.mk_term(
                            Kind.BV_XOR,
                            [gt1_ul, tm.mk_term(Kind.BV_XOR, [gt2_ul, gt3_ul])],
                        ),
                    ],
                ),
            ],
        )
    )

In [8]:
# create the constraints for the sbox output link the gate output
for i in range(bits):
    ts_temp = [tm.mk_term(Kind.EQUAL, [ys_v[i], ts_v[j]]) for j in range(gate_number)]
    bitwuzla.assert_formula(tm.mk_term(Kind.OR, ts_temp))

In [9]:
assertions = bitwuzla.get_assertions()
print('Assertions:')
print('{')
for a in assertions:
    print(f' {a}')
print('}')

Assertions:
{
 (= x0 #b0011)
 (= x1 #b0101)
 (= y0 #b0100)
 (= y1 #b0110)
 (or (= q0 x0) (= q0 x1))
 (or (= q1 x0) (= q1 x1))
 (or (or (= q2 x0) (= q2 x1)) (= q2 t0))
 (or (or (= q3 x0) (= q3 x1)) (= q3 t0))
 (or (= gt0 #b0110) (= gt0 #b0001))
 (or (= gt1 #b0110) (= gt1 #b0001))
 (= t0 (bvxor (ite (bvult ((_ extract 0 0) gt0) #b1) #b0000 (bvand q0 q1)) (bvxor (ite (bvult ((_ extract 1 1) gt0) #b1) #b0000 q1) (bvxor (ite (bvult ((_ extract 2 2) gt0) #b1) #b0000 q0) (ite (bvult ((_ extract 3 3) gt0) #b1) #b0000 #b1111)))))
 (= t1 (bvxor (ite (bvult ((_ extract 0 0) gt1) #b1) #b0000 (bvand q2 q3)) (bvxor (ite (bvult ((_ extract 1 1) gt1) #b1) #b0000 q3) (bvxor (ite (bvult ((_ extract 2 2) gt1) #b1) #b0000 q2) (ite (bvult ((_ extract 3 3) gt1) #b1) #b0000 #b1111)))))
 (or (= y0 t0) (= y0 t1))
 (or (= y1 t0) (= y1 t1))
}


In [10]:
result = bitwuzla.check_sat()

In [16]:
if result == Result.SAT:
    print('SAT')
    for x in xs_v:
        print(f'{x} = {bitwuzla.get_value(x)}')
    for q in qs_v:
        print(f'{q} = {bitwuzla.get_value(q)}')     
    for t in ts_v:
        print(f'{t} = {bitwuzla.get_value(t)}')
    for gt in gts_v:
        print(f'{gt} = {bitwuzla.get_value(gt)}')
    for y in ys_v:
        print(f'{y} = {bitwuzla.get_value(y)}')
    

SAT
x0 = #b0011
x1 = #b0101
q0 = #b0101
q1 = #b0011
q2 = #b0110
q3 = #b0101
t0 = #b0110
t1 = #b0100
gt0 = #b0110
gt1 = #b0001
y0 = #b0100
y1 = #b0110
